# MapReduce

A different paradigm for processing data

We have some data
<img src="http://www.gilliganondata.com/wp-content/uploads/2012/06/ExcelTables_1.png">


> What does it mean by “sharding” a set or database?

* Partition the database into smaller parts by using a column-based selection 
    - **Vertical partitioning**
* Partition the database into smaller parts by using a row-based selection 
    - **Horizontal partitioning**
* Partition the database into smaller parts by using a procedure
    - e.g. standard hash function: `k(x) -> n(x * p)`


There are many systems that take advantage 

of partinioning in computation

## **MapReduce** is a completely different paradigm


* Solving a certain subset of parallelizable problems
* It brings the compute to the data
* Input data is not stored on a separate storage system
    * Data exists in little pieces and is permanently stored on each computing node
    * distributed system

## History of implementations of the functional programming approach 
## on a distributed system

* Google File System (paper published in 2003)
* Google MapReduce (paper published in 2003 – implemented at Google in 2002)
* Hadoop (2006-2008) 
    - HDFS
    - MapReduce
    - A whole ecosystem

## GFS design

* Hardware failures are common
    - If medium-time-between-failure is 1 year – Then 10000 servers have one failure / hour
* Files are huge (GB) 
* Sequential writes
    - typically most files are mutated by appending newdata rather than overwriting
* Sequential reads
    - once written, the files are only read, and often only sequentially
* High sustained bandwidth

## Sharding/Chunking

- Files are divided into fixed-size chunks
- Size: typically 64/128 MB (modifiable parameter)
- Files are replicated (by default 3 times, fault-tolerance)

## Data replication

What’s the reason why a new data set gets replicated three times of different nodes?

- Performance
    * It improves the data locality
- Fault-tolerance
    * If a node crashes, its load can be moved onto another resource


Advantages of (large) fixed-size chunks:
  * Disk seek time small compared to transfer time
  * A single file can be larger than a node’s disk space   
  * Fixed size makes allocation computations easy

Why not increase the chunk size further?

- Maps task operate on one chunk at a time
- the increasing of the chunk size decreases the parallelism

## Write Operation

> If some replica returns a failure, 
*the offset value* is changed and the write process is restarted with the new value.

<center>
<img src="https://www.cs.rutgers.edu/~pxk/417/notes/images/dfs-1-gfs-chunks.png" width=800>
</center>

# Hadoop
(and Hadoop streaming)

<center>
<img src='http://www.opensourceforu.efytimes.com/wp-content/uploads/2012/03/hadoop-database-590x321.jpg'>
</center>

## Hadoop distributed file system

HDFS stores large files (typically in the range of gigabytes to terabytes) across multiple machines. 

It achieves reliability by replicating the data across multiple hosts, and hence theoretically does not require RAID storage on hosts.

A Hadoop cluster has nominally a single namenode plus a cluster of datanodes

- Each datanode serves up blocks of data over the network using a block protocol specific to HDFS. 
- The file system uses TCP/IP sockets for communication. 
- Clients use remote procedure call (RPC) to communicate between each other.

## What you receive from an Hadoop environment

* Partitioning the input data
* Scheduling the program’s execution across a set of machines
* Performing the group by key step
* Handling node failures
* Managing required inter-machine communication

> Ok. What is MapReduce again?

### Map function

processes data and generates a set of intermediate key/value pairs.

### Reduce function

merges all intermediate values associated with the same intermediate key.

<center>
<img src="images/mapreduce.png" width=600>
</center>

# Word count
The '`Hello World`' for MapReduce (with *Java*)

Among the simplest of full Hadoop jobs you can run

<img src='http://www.glennklockwood.com/data-intensive/hadoop/wordcount-schematic.png'
width='700'>
<small>Reading ***Moby Dick*** </small>

Consider doing a word count of the following file using  MapReduce:
```
Hello World Bye World
Hello Hadoop Goodbye Hadoop
```

The map function reads in words one at a time outputs (“word”, 1) for each parsed input word

```
(Hello, 1)
(World, 1)
(Bye, 1)
(World, 1)
(Hello, 1)
(Hadoop, 1)
(Goodbye, 1)
(Hadoop, 1)
```

The shuffle phase between map and reduce creates a  list of values associated with each key
```
(Bye, (1))
(Goodbye, (1))
(Hadoop, (1, 1))
(Hello, (1, 1))
(World, (1, 1))
```

The reduce function sums the numbers in the list for each  key and outputs (word, count) pairs
```
(Bye, 1)
(Goodbye, 1)
(Hadoop, 2)
(Hello, 2)
(World, 2)
```

## How can you do this with Java?
(the Hadoop framework native language)

``` Java
// Imports
package org.myorg;
import java.io.IOException;
import java.util.*;
import org.apache.hadoop.*

// Create JAVA class
public class WordCount {
```

``` Java
//Mapper function
  public static class Map extends MapReduceBase implements Mapper<LongWritable, Text, Text, IntWritable> {
    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(LongWritable key, Text value, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {
      String line = value.toString();
      StringTokenizer tokenizer = new StringTokenizer(line);
      while (tokenizer.hasMoreTokens()) {
        word.set(tokenizer.nextToken());
        output.collect(word, one);
      }
    }
  }
```

``` Java
//Reducer function
  public static class Reduce extends MapReduceBase implements Reducer<Text, IntWritable, Text, IntWritable> {
    public void reduce(Text key, Iterator<IntWritable> values, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {
      int sum = 0;
      while (values.hasNext()) {
        sum += values.next().get();
      }
      output.collect(key, new IntWritable(sum));
    }
  }
    
```

<small>
``` Java
//Main function
  public static void main(String[] args) throws Exception {
    JobConf conf = new JobConf(WordCount.class);
    conf.setJobName("wordcount");

    conf.setOutputKeyClass(Text.class);
    conf.setOutputValueClass(IntWritable.class);

    conf.setMapperClass(Map.class);
    conf.setCombinerClass(Reduce.class);
    conf.setReducerClass(Reduce.class);

    conf.setInputFormat(TextInputFormat.class);
    conf.setOutputFormat(TextOutputFormat.class);

    FileInputFormat.setInputPaths(conf, new Path(args[0]));
    FileOutputFormat.setOutputPath(conf, new Path(args[1]));

    JobClient.runJob(conf);
  }
```
</small>

We can test the Java code here. *Live*.

In [1]:
%env HADOOP_EXAMPLES /usr/local/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.6.0.jar

env: HADOOP_EXAMPLES=/usr/local/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.6.0.jar


In [3]:
# Hadoop available examples
! hadoop jar $HADOOP_EXAMPLES | grep word

  aggregatewordcount: An Aggregate based map/reduce program that counts the words in the input files.
  aggregatewordhist: An Aggregate based map/reduce program that computes the histogram of the words in the input files.
  multifilewc: A job that counts words from several files.
  wordcount: A map/reduce program that counts the words in the input files.
  wordmean: A map/reduce program that counts the average length of the words in the input files.
  wordmedian: A map/reduce program that counts the median length of the words in the input files.
  wordstandarddeviation: A map/reduce program that counts the standard deviation of the length of the words in the input files.


In [4]:
# Check wordcount
! hadoop jar $HADOOP_EXAMPLES wordcount

Usage: wordcount <in> [<in>...] <out>


## Demo

Note to my self: run next cells 'live'

In [ ]:
%env HADOOP_EXAMPLES /usr/local/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.6.0.jar

In [ ]:
# Our input
! cat ../data/books/twolines.txt

In [ ]:
%%bash

########################
# Preprocess with HDFS

# Create input directory
hdfs dfs -mkdir myinput
# Save one file inside
file="../data/books/twolines.txt"
hdfs dfs -put $file myinput/file01
# Remove output or Hadoop will give error if existing
hdfs dfs -rm -r -f myoutput

In [ ]:
# Test wordcount with real hadoop on our system
! hadoop jar $HADOOP_EXAMPLES wordcount myinput myoutput

In [ ]:
# Hadoop output
! hadoop fs -cat myoutput/part*

# Hadoop streaming
### Concepts and mechanisms

Hadoop streaming is a utility 

* It comes bundled with the Hadoop distribution
* It allows creating and running Map/Reduce jobs 
    - with any executable or script as the mapper and/or the reducer

Protocol steps

* Create a Map/Reduce job
* Submit the job to an appropriate cluster
* Monitor the progress of the job until it completes
* Links to Hadoop HDFS job directories

### Why?

One of the most unappetizing aspects of Hadoop to users of traditional HPC is that it is written in Java. 

* Java is not originally designed to be a high-performance language
* Learning Java is very difficult for domain scientists

This is why Hadoop allows you to write map/reduce code in any language you want using the Hadoop Streaming interface

* It means turning an existing Python or Perl script into a Hadoop job
* Does not require learning any Java at all

A streaming command line example **for python**:
``` bash
$ hadoop jar $HADOOP_HOME/hadoop-streaming.jar \
    -files mapper.py,reducer.py
    -input input_dir/ \
    -output output_dir/ \
    -mapper mapper.py \
    -reducer reducer.py \
```

Before submitting the Hadoop Streaming job:

* Make sure your scripts have no errors
* Do mapper and reducer scripts actually work?

This is just a matter of running them through pipes on a **little bit** of sample data,

like `cat` or `head` linux bash commands, with pipes, as seen before.

```
# Simulating hadoop streaming with bash pipes
$ cat $file | python mapper.py | sort | python reducer.py
```

In [6]:
%%writefile mapper.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
TAB = "\t"
SEP = ':'
import sys

# Cycle current streaming data
for line in sys.stdin:
    # Clean input
    line = line.strip()
    # Skip SAM/BAM headers
    if line[0] == "@":
        continue
    
    # Use data
    pieces = line.split(TAB)
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]

    mystop = mystart + len(myseq)

    # Each element with coverage
    for i in range(mystart,mystop):
        results = [mychr+SEP+i.__str__(), "1"]
        print(TAB.join(results))


Overwriting mapper.py


In [7]:
%%writefile reducer.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
TAB = "\t"
SEP = ':'
import sys
last_value = ""
value_count = 1
for line in sys.stdin:
    value, count = line.strip().split(TAB)
    # if this is the first iteration
    if not last_value:
        last_value = value
    # if they're the same, log it
    if value == last_value:
        value_count += int(count)
    else:
        # state change
        try: 
            print(TAB.join([last_value, str(value_count)]))
        except:
            pass
        last_value = value
        value_count = 1
# LAST ONE after all records have been received
print(TAB.join([last_value, str(value_count)]))

Overwriting reducer.py


In [24]:
%%bash
# needs ~ 5 seconds for running
time head -n 10000 $myfile | python mapper.py | sort | python reducer.py | head -n 5

chr1:10000	3
chr1:10001	2
chr1:10002	2
chr1:10003	2
chr1:10004	2



real	0m3.880s
user	0m3.800s
sys	0m0.070s


<big>
A working python code tested on pipes **should work** with Hadoop Streaming
</big>

* To make this happen we need to handle copy of input and output files inside the Hadoop FS
* Also the job tracker logs will be found inside HDFS
* We are going to use bash scripting inside the notebook to make our workflow

## Preprocessing

HDFS commands to interact with Hadoop file system use the same syntax:

```
hdfs dfs -command
```

`command` are like bash commands for file

e.g.

```
hadoop fs -mkdir hdfs:///dir
hadoop fs -put file_on_host hdfs:///path/to/file
hadoop fs -ls
```

<small>
Note: we have seen this in action with the Java example
</small>

<big>
Hadoop Streaming needs “binaries” to execute
</big>

You need to specify interpreter at the beginning of your scripts:
```
#!/usr/bin/env python
```

Make also the script executables:
```
chmod +x hs*.py
```

In [27]:
! chmod +x mapper.py reducer.py

In [2]:
%env HADOOP_STREAMING /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar

env: HADOOP_STREAMING=/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar


In [29]:
%%bash
# Launch streaming
hadoop jar $HADOOP_STREAMING

Usage: $HADOOP_PREFIX/bin/hadoop jar hadoop-streaming.jar [options]
Options:
  dumptb <glob-pattern> Dumps all files that match the given pattern to 
                        standard output as typed bytes.
  loadtb <path> Reads typed bytes from standard input and stores them in
                a sequence file in the specified path
  [streamjob] <args> Runs streaming job with given arguments


No Arguments Given!


In [5]:
%%bash
# Preprocess with HDFS
hdfs dfs -rm -r -f myinput
hdfs dfs -mkdir myinput
# Save one file inside
hdfs dfs -put /tmp/ngs.sam myinput/file01
# Remove output or Hadoop will give error if existing
hdfs dfs -rm -r -f myoutput

Deleted myinput


16/03/15 17:04:11 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


Final launch via bash command for using Hadoop streaming

In [ ]:
%%bash 
# A real Hadoop Streaming run
time hadoop jar $HADOOP_STREAMING \
    -files mapper.py,reducer.py \
    -input myinput -output myoutput \
    -mapper mapper.py -reducer reducer.py

Hadoop streaming is **difficult to debug**.
Just like real Java Hadoop.

If you did a typical setup mistake, you may end receiving unrelated errors stacktrace from the Java virtual machine.

So before googling those stacktrace, make sure that:

* Python files (mapper and reducer) exists
* They are provided inside the main bash command also as **files** list
* They are executables and contain as first line the hashbang
* Your input directory exists on HDFS
* Files inside your input directory are not corrupted
    - e.g. bad decompression

## CAP theorem

* Consistency
    - all nodes see the same data at the same time
* Availability 
    - a guarantee that every request receives a response about whether it succeeded or failed
* Partition tolerance 
    - the system continues to operate despite arbitrary partitioning due to network failures

<img src="http://j.mp/1S7i4Eo">

# End of Chapter

In [1]:
%load_ext watermark
%watermark -a "Paolo D." -d -v -m -p jupyter

Paolo D. 2016-04-12 

CPython 3.5.1
IPython 4.1.2

jupyter 1.0.0

compiler   : GCC 4.4.7 20120313 (Red Hat 4.4.7-1)
system     : Linux
release    : 4.3.3-dhyve
machine    : x86_64
processor  : 
CPU cores  : 1
interpreter: 64bit
